这一章我们要学习的方法是贝叶斯，通过贝叶斯方法，猜测用户输入有误的单词应该是什么

公式如下：
argmaxc P(c|w) -> argmaxc P(w|c) P(c) / P(w)

这里，P(c)表示c出现的概率，P(w|c)表示想要输入c但错误输入w的概率


关键词：贝叶斯，概率

In [1]:
import re, collections

# 提取所有单词，都变成小写
def get_words(text):
    return re.findall('[a-z]+', text.lower())

# 统计每个字母出现在数据集中的的次数
def count_words(words):
    # 因为这里是从词库里面统计单词，如果输入的单词是正确的，但在词库里没有这个词
    # 这时候单词概率是0，这样会导致计算出问题，所以需要设为最小概率，所以用lambda: 1
    model = collections.defaultdict(lambda: 1)
    for f in words:
        model[f] += 1
    return model

# 读取文件，计算字母出现频率
wordslib = count_words(get_words(open('big.txt').read()))
wordslib

defaultdict(<function __main__.count_words.<locals>.<lambda>>,
            {'the': 80031,
             'project': 289,
             'gutenberg': 264,
             'ebook': 88,
             'of': 40026,
             'adventures': 18,
             'sherlock': 102,
             'holmes': 468,
             'by': 6739,
             'sir': 178,
             'arthur': 35,
             'conan': 5,
             'doyle': 6,
             'in': 22048,
             'our': 1067,
             'series': 129,
             'copyright': 70,
             'laws': 234,
             'are': 3631,
             'changing': 45,
             'all': 4145,
             'over': 1283,
             'world': 363,
             'be': 6156,
             'sure': 124,
             'to': 28767,
             'check': 39,
             'for': 6940,
             'your': 1280,
             'country': 424,
             'before': 1364,
             'downloading': 6,
             'or': 5353,
             'redistributing': 8,
       

因为我们是对比单词去判断，这里可以采用两个单词的距离
我们可以设想4种方法：
1. 在单词中增加一个字母
2. 在单词中删除一个字母
3. 交换单词中相邻的两个字母
4. 替换单词中的一个字母

In [2]:
word = 'jupyter'
n = len(word)
alphabet = 'abcdefghijklmnopqrstuvwxyz'

print('删除字母')
print(set([word[0:i] + word[i + 1:] for i in range(n)]))
for i in range(n):
    print('0: ', word[0:i], '1: ', word[i + 1:])
      
print('交换字母位置')
print(set([word[0:i] + word[i + 1] + word[i] + word[i + 2:] for i in range(n-1)]))
for i in range(n - 1):
    print('0: ', word[0:i], '1: ', word[i + 1], '2: ', word[i], '3: ', word[i + 2:])

print('替换字母')
print(set([word[0:i] + c + word[i + 1:] for i in range(n) for c in alphabet]))
for i in range(n):
    print('round: ', i)
    for c in alphabet:
        print('0: ', word[0:i], 'c: ', c, '1: ', word[i + 1:])

print('增加字母')
print(set([word[0:i] + c + word[i:] for i in range(n + 1) for c in alphabet]))
for i in range(n + 1):
    print('round: ', i)
    for c in alphabet:
        print('0: ', word[0:i], 'c: ', c, '1: ', word[i:])

删除字母
{'juyter', 'jupyer', 'jpyter', 'jupytr', 'jupter', 'upyter', 'jupyte'}
0:   1:  upyter
0:  j 1:  pyter
0:  ju 1:  yter
0:  jup 1:  ter
0:  jupy 1:  er
0:  jupyt 1:  r
0:  jupyte 1:  
交换字母位置
{'juptyer', 'juypter', 'jupytre', 'jpuyter', 'ujpyter', 'jupyetr'}
0:   1:  u 2:  j 3:  pyter
0:  j 1:  p 2:  u 3:  yter
0:  ju 1:  y 2:  p 3:  ter
0:  jup 1:  t 2:  y 3:  er
0:  jupy 1:  e 2:  t 3:  r
0:  jupyt 1:  r 2:  e 3:  
替换字母
{'bupyter', 'jupzter', 'jupster', 'jupytsr', 'jupytcr', 'jdpyter', 'xupyter', 'jupfter', 'juptter', 'jupytdr', 'jugyter', 'jupdter', 'juprter', 'japyter', 'jupyper', 'jupytpr', 'jkpyter', 'jvpyter', 'jupytev', 'jopyter', 'jxpyter', 'juayter', 'jukyter', 'iupyter', 'juppter', 'jupyuer', 'jupytvr', 'juwyter', 'juqyter', 'jupnter', 'yupyter', 'jupyqer', 'jupqter', 'jupyteu', 'judyter', 'jupytrr', 'jupytem', 'lupyter', 'jutyter', 'pupyter', 'gupyter', 'jupyier', 'jupytei', 'jupytes', 'jupyten', 'jwpyter', 'jupytex', 'jupytnr', 'vupyter', 'juryter', 'jzpyter', 'jupvter'

0:   c:  r 1:  jupyter
0:   c:  s 1:  jupyter
0:   c:  t 1:  jupyter
0:   c:  u 1:  jupyter
0:   c:  v 1:  jupyter
0:   c:  w 1:  jupyter
0:   c:  x 1:  jupyter
0:   c:  y 1:  jupyter
0:   c:  z 1:  jupyter
round:  1
0:  j c:  a 1:  upyter
0:  j c:  b 1:  upyter
0:  j c:  c 1:  upyter
0:  j c:  d 1:  upyter
0:  j c:  e 1:  upyter
0:  j c:  f 1:  upyter
0:  j c:  g 1:  upyter
0:  j c:  h 1:  upyter
0:  j c:  i 1:  upyter
0:  j c:  j 1:  upyter
0:  j c:  k 1:  upyter
0:  j c:  l 1:  upyter
0:  j c:  m 1:  upyter
0:  j c:  n 1:  upyter
0:  j c:  o 1:  upyter
0:  j c:  p 1:  upyter
0:  j c:  q 1:  upyter
0:  j c:  r 1:  upyter
0:  j c:  s 1:  upyter
0:  j c:  t 1:  upyter
0:  j c:  u 1:  upyter
0:  j c:  v 1:  upyter
0:  j c:  w 1:  upyter
0:  j c:  x 1:  upyter
0:  j c:  y 1:  upyter
0:  j c:  z 1:  upyter
round:  2
0:  ju c:  a 1:  pyter
0:  ju c:  b 1:  pyter
0:  ju c:  c 1:  pyter
0:  ju c:  d 1:  pyter
0:  ju c:  e 1:  pyter
0:  ju c:  f 1:  pyter
0:  ju c:  g 1:  pyter
0:  ju c:  h 1

In [3]:
# 整理上面的方法，把4种结果结合到一个set里，作为一个单词的猜词可能集
def edits1(word):
    n = len(word)
    return set([word[0:i] + word[i + 1:] for i in range(n)] +                     # 遍历删除某个字母
               [word[0:i] + word[i + 1] + word[i] + word[i + 2:] for i in range(n-1)] + # 遍历交替字母位置
               [word[0:i] + c + word[i + 1:] for i in range(n) for c in alphabet] + # 遍历替换字母
               [word[0:i] + c + word[i:] for i in range(n + 1) for c in alphabet])  # 遍历增加字母

上面这样通过字母的变动，变成另一个“单词”，这称为两个单词的编辑距离
变动一个字母称为编辑距离为1，如果是变动两个字母，则为2

In [4]:
# 我们再设置一个编辑距离为2的集
def edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1))

In [5]:
a = set()
for i in edits1('something'):
    for j in edits1(i):
        a.add(j)
print(len(a))

114324


现在出现一个问题，就是如果通过这样的一个字母的更改，会出现很多个词，甚至是不存在的单词
当编辑距离再增大的时候，会返回更多“单词”
这时候我们考虑，返回那些在词库里出现过的正确单词
当出现多个结果的时候，我们可以优先考虑距离最近的正确单词，比如编辑距离1的就比编辑距离2的可能性大

In [6]:
def get_correct_words(words):
    return set(w for w in words if w in wordslib)

def guess(word):
    # 下面按优先顺序，先把单词放到词库去查找，找到就直接返回，不继续查找后面的
    # 第一次没有找到，则按编辑距离1去获取可能词集，再去词库查找
    # 再来按编辑距离2去找
    # 如果都没有找到，最后返回单词本身
    candidates = get_correct_words([word]) or get_correct_words(edits1(word)) or get_correct_words(edits2(word)) or [word]
    # 结果会返回最大概率的单词
    return max(candidates, key=lambda w: wordslib[w])

In [7]:
# 最后我们来测试看看效果

guess('soethin')

'something'

在这一章节里，我们学习到了贝叶斯概念：通过已知先验概率，预估最大可能